# Performance

This notebook measures performance of `Simulate.jl` functionality in order to 

1. improve it,
2. get a ground for decisions about issue ["Getting rid of all functionality involving gobal scope #15"](https://github.com/pbayer/Simulate.jl/issues/15) and
3. compile the [performance section](https://pbayer.github.io/Simulate.jl/dev/performance/) of the documentation.

In [1]:
using Simulate, BenchmarkTools, Random
res = Dict(); # results dictionary

## Event-based simulations

The following is a modification of the [channel example](https://pbayer.github.io/Simulate.jl/dev/approach/#Event-based-modeling-1). We simulate events 

1. taking something from a common channel or waiting if there is nothing, 
2. then taking a delay, doing a calculation and
3. returning three times to the first step.

As calculation we take the following Machin-like sum:

$$4 \sum_{k=1}^{n} \frac{(-1)^{k+1}}{2 k - 1}$$

This gives a slow approximation to $\pi$. 

### Function calls as events

The first implementation is based on events with `SimFunction`s. 

In [2]:
function take(id::Int64, ch::Channel, step::Int64, qpi::Array{Float64,1})
    if isready(ch)
        take!(ch)                                            # take something from common channel
        event!(SF(put, id, ch, step, qpi), after, rand())    # timed event after some time
    else
        event!(SF(take, id, ch, step, qpi), SF(isready, ch)) # conditional event until channel is ready
    end
end

function put(id::Int64, ch::Channel, step::Int64, qpi::Array{Float64,1})
    put!(ch, 1)
    qpi[step] += (-1)^(id+1)/(2id -1)      # Machin-like series (slow approximation to pi)
    step > 3 || take(id, ch, step+1, qpi)
end

function setup(n::Int)                     # a setup he simulation
    reset!(𝐶)
    Random.seed!(123)
    global ch = Channel{Int64}(32)  # create a channel
    global qpi = zeros(4)
    si = shuffle(1:n)
    for i in 1:n
        take(si[i], ch, 1, qpi)
    end
    for i in 1:min(n, 32)
        put!(ch, 1) # put first tokens into channel 1
    end
end

setup (generic function with 1 method)

If we setup 250 summation elements, we get 1000 timed events and over 1438 sample steps with conditional events.

In [4]:
@time setup(250)
println(@time run!(𝐶, 500))
println("result=", sum(qpi))

  0.009447 seconds (2.55 k allocations: 153.891 KiB)
  0.397957 seconds (2.48 M allocations: 115.761 MiB, 6.75% gc time)
run! finished with 1000 clock events, 1438 sample steps, simulation time: 500.0
result=3.137592669589475


In [5]:
t = run(@benchmarkable run!(𝐶, 500) setup=setup(250) evals=1 seconds=10.0 samples=30)

BenchmarkTools.Trial: 
  memory estimate:  115.75 MiB
  allocs estimate:  2481210
  --------------
  minimum time:     351.639 ms (2.16% GC)
  median time:      357.934 ms (3.11% GC)
  mean time:        359.288 ms (2.85% GC)
  maximum time:     374.970 ms (3.21% GC)
  --------------
  samples:          27
  evals/sample:     1

In [6]:
res["Event based with SimFunctions"] = minimum(t).time * 1e-6 # ms 

351.639478

### Expressions as events

The 2nd implementation does the same but with expressions, which are `eval`uated in global scope during runtime:

In [7]:
function take(id::Int64, ch::Channel, step::Int64, qpi::Array{Float64,1})
    if isready(ch)
        take!(ch)                                            # take something from common channel
        event!(:(put($id, ch, $step, qpi)), after, rand())   # timed event after some time
    else
        event!(:(take($id, ch, $step, qpi)), :(isready(ch))) # conditional event until channel is ready
    end
end

function put(id::Int64, ch::Channel, step::Int64, qpi::Array{Float64,1})
    put!(ch, 1)
    qpi[step] += (-1)^(id+1)/(2id -1)      # Machin-like series (slow approximation to pi)
    step > 3 || take(id, ch, step+1, qpi)
end

put (generic function with 1 method)

In [8]:
@time setup(250)
println(@time run!(𝐶, 500))
println("result=", sum(qpi))

  0.087837 seconds (158.23 k allocations: 8.476 MiB)
 12.844018 seconds (7.92 M allocations: 487.178 MiB, 0.50% gc time)
run! finished with 1000 clock events, 1438 sample steps, simulation time: 500.0
result=3.137592669589475


In [9]:
t = run(@benchmarkable run!(𝐶, 500) setup=setup(250) evals=1 seconds=10.0 samples=30)

BenchmarkTools.Trial: 
  memory estimate:  486.70 MiB
  allocs estimate:  7905745
  --------------
  minimum time:     12.834 s (0.44% GC)
  median time:      12.834 s (0.44% GC)
  mean time:        12.834 s (0.44% GC)
  maximum time:     12.834 s (0.44% GC)
  --------------
  samples:          1
  evals/sample:     1

In [10]:
res["Event based with Expressions"] = minimum(t).time * 1e-6 #
res

Dict{Any,Any} with 2 entries:
  "Event based with Expressions"  => 12833.6
  "Event based with SimFunctions" => 351.639

In [11]:
res["Event based with Expressions"]/res["Event based with SimFunctions"]

36.49634252386189

This takes much longer and shows that `eval` for Julia expressions, done in global scope is very expensive and should be avoided if performance is any issue.

### Involving a global variable

The third implementation works with `Simfunction`s like the first but involves a global variable `A`:

In [12]:
function take(id::Int64, ch::Channel, step::Int64)::Float64
    if isready(ch)
        take!(ch)                                       # take something from common channel
        event!(SF(put, id, ch, step), after, rand())    # timed event after some time
    else
        event!(SF(take, id, ch, step), SF(isready, ch)) # conditional event until channel is ready
    end
end

function put(id::Int64, ch::Channel, step::Int64)
    put!(ch, 1)
    global A += (-1)^(id+1)/(2id -1)      # Machin-like series (slow approximation to pi)
    step > 3 || take(id, ch, step+1)
end

function setup(n::Int)                     # a setup he simulation
    reset!(𝐶)
    Random.seed!(123)
    global ch = Channel{Int64}(32)  # create a channel
    global A = 0
    si = shuffle(1:n)
    for i in 1:n
        take(si[i], ch, 1)
    end
    for i in 1:min(n, 32)
        put!(ch, 1) # put first tokens into channel 1
    end
end

setup (generic function with 1 method)

In [13]:
ch = Channel{Int64}(32)
@code_warntype put(1, ch, 1)

Variables
  #self#::Core.Compiler.Const(put, false)
  id::Int64
  ch::Channel{Int64}
  step::Int64

Body::Union{Bool, Float64}
1 ─       Main.put!(ch, 1)
│         nothing
│   %3  = (id + 1)::Int64
│   %4  = ((-1) ^ %3)::Int64
│   %5  = (2 * id)::Int64
│   %6  = (%5 - 1)::Int64
│   %7  = (%4 / %6)::Float64
│   %8  = (Main.A + %7)::Any
│         (Main.A = %8)
│   %10 = (step > 3)::Bool
└──       goto #3 if not %10
2 ─       return %10
3 ─ %13 = (step + 1)::Int64
│   %14 = Main.take(id, ch, %13)::Float64
└──       return %14


In [15]:
@time setup(250)
println(@time run!(𝐶, 500))
println("result=", A)

  0.007132 seconds (3.05 k allocations: 157.688 KiB)
  0.366033 seconds (2.48 M allocations: 115.757 MiB, 3.80% gc time)
run! finished with 1000 clock events, 1438 sample steps, simulation time: 500.0
result=3.1375926695894556


In [16]:
t = run(@benchmarkable run!(𝐶, 500) setup=setup(250) evals=1 seconds=10.0 samples=30)

BenchmarkTools.Trial: 
  memory estimate:  115.75 MiB
  allocs estimate:  2483207
  --------------
  minimum time:     357.921 ms (2.33% GC)
  median time:      364.239 ms (3.53% GC)
  mean time:        365.695 ms (3.14% GC)
  maximum time:     382.753 ms (2.26% GC)
  --------------
  samples:          27
  evals/sample:     1

In [17]:
res["Event based with functions and a global variable"] = minimum(t).time * 1e-6 #
res

Dict{Any,Any} with 3 entries:
  "Event based with Expressions"                     => 12833.6
  "Event based with SimFunctions"                    => 351.639
  "Event based with functions and a global variable" => 357.921

In this case the compiler does well to infer the type of `A` and it takes only marginally longer than the first version.